In [1]:
import turicreate as tc

In [3]:
textfile = tc.SArray('text.txt')

Read 1 lines. Lines per second: 19.181

Finished parsing file /home/jovyan/work/ncbi_data_test/metadata-wizard/text.txt

Parsing completed. Parsed 1 lines in 0.060425 secs.

In [8]:
text = tc.text_analytics.count_words(textfile)
text = text.dict_trim_by_keys(tc.text_analytics.stop_words(), True)

In [9]:
tokens = tc.text_analytics.tokenize(text)

NameError: name 'sa' is not defined

In [12]:
topic_model = tc.topic_model.create(text, num_topics = verbose = False)

Learning a topic model

Number of documents         1

Vocabulary size      7908

Running collapsed Gibbs sampling

+-----------+---------------+----------------+-----------------+

| Iteration | Elapsed Time  | Tokens/Second  | Est. Perplexity |

+-----------+---------------+----------------+-----------------+

| 10        | 51.196ms      | 3.60766e+06    | 0               |

+-----------+---------------+----------------+-----------------+

In [14]:
text_topics = topic_model.get_topics()

In [16]:
text_topics.print_rows(50,3)

+-------+---------------+----------------------+
| topic |      word     |        score         |
+-------+---------------+----------------------+
|   0   |    patients   | 0.08974397772667314  |
|   0   |     genes     | 0.017567485776541478 |
|   0   |     levels    | 0.017416172376226738 |
|   0   |      data     | 0.01681091877496777  |
|   0   | characterized | 0.016054351773394065 |
|   1   |     normal    | 0.04091241889469816  |
|   1   |    analysis   | 0.03674669898255673  |
|   1   |    protein    | 0.03649423110909361  |
|   1   |    families   | 0.03548435961524114  |
|   1   |     human     | 0.03093993789290503  |
|   2   |     cells     | 0.023551461614460704 |
|   2   |     allele    | 0.023406171906783586 |
|   2   |    linkage    | 0.02166269541465818  |
|   2   |      cell     | 0.021372115999303942 |
|   2   |     breast    | 0.020209798337887008 |
|   3   |      gene     | 0.13298656618611487  |
|   3   |    disease    | 0.04806957186544611  |
|   3   |       5   

In [17]:
#Load the data
data = tc.SFrame('diseases_all_consensus.csv')

Finished parsing file /home/jovyan/work/ncbi_data_test/metadata-wizard/diseases_all_consensus.csv

Parsing completed. Parsed 100 lines in 0.05071 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 2136 lines. Lines per second: 58040.3

Finished parsing file /home/jovyan/work/ncbi_data_test/metadata-wizard/diseases_all_consensus.csv

Parsing completed. Parsed 2136 lines in 0.037314 secs.

In [18]:
data[0]

{'value': 'A-T', 'synthetic_class': 'Modifer'}

In [19]:
disease_classes = [i for i in data['synthetic_class'].unique()]

In [20]:
disease_classes

['Modifer', 'CompositeMention', 'SpecificDisease', 'DiseaseClass']

### Create Feature using topic modeling

In [49]:
# Remove stopwords and convert to bag of words
    doc = tc.text_analytics.count_words(data['value'])
    doc = doc.dict_trim_by_keys(tc.text_analytics.stop_words(), True)

In [50]:

# Learn topic model
    model = tc.topic_model.create(doc, initial_topics=text_topics['word'], verbose=False)
    # Agreaggate the unique words
    sf_topics = model.get_topics()
    sf_topics = sf_topics.append(text_topics)
    sf_words = sf_topics.groupby(key_column_names='word', operations={'sum_scores': tc.aggregate.SUM('score')})
    
    # Sort the features scores and filter out all those which are key
    sf_words = sf_words.sort('sum_scores', ascending= False).filter_by(disease_classes, 'word', exclude=True)

Learning a topic model

Number of documents      2136

Vocabulary size      1548

Running collapsed Gibbs sampling

+-----------+---------------+----------------+-----------------+

| Iteration | Elapsed Time  | Tokens/Second  | Est. Perplexity |

+-----------+---------------+----------------+-----------------+

| 10        | 51.304ms      | 1.28509e+06    | 0               |

+-----------+---------------+----------------+-----------------+

In [51]:
sf_words.print_rows(50,2)

+---------------+----------------------+
|      word     |      sum_scores      |
+---------------+----------------------+
|   deficiency  | 0.32689396380620717  |
|    disease    | 0.19947908884767843  |
|     cancer    |  0.1694339969898547  |
|    syndrome   | 0.13823812636633295  |
|      gene     | 0.13298656618611487  |
|   mutations   | 0.10289367246668593  |
|    patients   | 0.08974397772667314  |
|    disorder   |   0.08867149980385   |
|     breast    | 0.08704441600237912  |
|    mutation   | 0.08197414357408278  |
|    ovarian    | 0.07755154556273806  |
|   autosomal   | 0.05758904879220308  |
|      cell     |  0.0566679566554252  |
|    cancers    | 0.05149346608587376  |
|      type     | 0.047253634894990806 |
|       1       | 0.044507834252336446 |
|     linked    | 0.04321486268174373  |
|     found     | 0.04243754419043407  |
|   deficient   | 0.041274669320536556 |
|     normal    | 0.04091241889469816  |
|   hereditary  | 0.040603609209707105 |
|   congenital  

In [52]:
def create_features(category_df, n_features):
    # Remove stopwords and convert to bag of words
    doc = tc.text_analytics.count_words(data['value'])
    doc = doc.dict_trim_by_keys(tc.text_analytics.stop_words(), True)
    
    # Learn topic model
    model = tc.topic_model.create(doc, initial_topics=text_topics['word'], verbose=False)
    # Agreaggate the unique words
    sf_topics = model.get_topics()
    #append the topics from abstract text
    
    sf_topics = sf_topics.append(text_topics)
    sf_words = sf_topics.groupby(key_column_names='word', operations={'sum_scores': tc.aggregate.SUM('score')})
    
    # Sort the features scores and filter out all those which are key
    sf_words = sf_words.sort('sum_scores', ascending= False).filter_by(disease_classes, 'word', exclude=True)
    
    # Take a look of the features related with this key class
    features = [i for i in sf_words['word']][0:n_features] #changable
    return sf_words, features

In [53]:
sf_words, features = create_features(data['value'], len(sf_words))

Learning a topic model

Number of documents      2136

Vocabulary size      1548

Running collapsed Gibbs sampling

+-----------+---------------+----------------+-----------------+

| Iteration | Elapsed Time  | Tokens/Second  | Est. Perplexity |

+-----------+---------------+----------------+-----------------+

| 10        | 42.422ms      | 2.92175e+06    | 0               |

+-----------+---------------+----------------+-----------------+

### Make input matrix

In [54]:
def get_input_matrix(features, category_df, sf_words):
    tuples = []
    for word in features:
        feature_vector = [1 if (word in i) else 0 for i in data['value']]
        tuples.append((word, feature_vector))
        
    sf_features = tc.SFrame({key: value for (key, value) in tuples})
    #concatenating the features with the category matrix
    category_df = category_df.add_row_number()
    sf_features = sf_features.add_row_number()
    final_table = category_df.join(sf_features, on='id', how='left')
    for f in features:
        score = sf_words[sf_words['word'] == str(f)]['sum_scores'].astype(float)[0]
        final_table[str(f)] = [(1.0+score) * i for i in final_table[str(f)]]
        
    return final_table

In [55]:
input_matrix = get_input_matrix(features, data, sf_words)

In [56]:
input_matrix

id,value,synthetic_class,0,1,2,3,5,6,abnormalities,abnormality
0,A-T,Modifer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,AAPC,Modifer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,AAS,Modifer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ACG1B,Modifer,0.0,1.0729458648126076,0.0,0.0,0.0,0.0,0.0,0.0
4,ACTH deficiency,SpecificDisease,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,AD,SpecificDisease,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,ADCA type I,Modifer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,ADNDI,SpecificDisease,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,ADO,Modifer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,AFAP,SpecificDisease,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [57]:
input_matrix.export_csv('feature_matrix_text_topics.csv')

### Multiclass classification

In [12]:
# Make a train-test split
train_data, test_data = input_matrix.random_split(0.7)

In [14]:
# Create a model.
model = tc.random_forest_classifier.create(train_data, target='Class',
                                           features = features,
                                           max_iterations=3,
                                           max_depth = 3)

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



External memory mode: 1 batches

External memory mode: 1 batches

Random forest classifier:

--------------------------------------------------------

...

+-----------+--------------+-------------------+---------------------+-------------------+---------------------+

External memory mode: 1 batches

External memory mode: 1 batches

External memory mode: 1 batches

External memory mode: 1 batches

In [15]:
# Save predictions to an SArray
predictions = model.predict(test_data)

External memory mode: 1 batches

In [16]:
# Evaluate the model and save the results into a dictionary
results = model.evaluate(test_data)

External memory mode: 1 batches

In [17]:
results.items()

dict_items([('accuracy', 0.5770925110132159), ('auc', 0.6374495122516017), ('confusion_matrix', Columns:
	target_label	str
	predicted_label	str
	count	int

Rows: 8

Data:
+------------------+------------------+-------+
|   target_label   | predicted_label  | count |
+------------------+------------------+-------+
|   DiseaseClass   | SpecificDisease  |  120  |
| SpecificDisease  | CompositeMention |   1   |
|     Modifier     | SpecificDisease  |   47  |
| SpecificDisease  | SpecificDisease  |  241  |
|     Modifier     | CompositeMention |   1   |
|   DiseaseClass   |   DiseaseClass   |   19  |
| CompositeMention | SpecificDisease  |   23  |
| CompositeMention | CompositeMention |   2   |
+------------------+------------------+-------+
[8 rows x 3 columns]
), ('f1_score', 0.273658375074505), ('log_loss', 1.0691241907913878), ('precision', 0.6863882443928847), ('recall', 0.30313960401926393), ('roc_curve', Columns:
	threshold	float
	fpr	float
	tpr	float
	p	int
	n	int
	class	int

Rows: 

In [18]:
results.keys()

dict_keys(['accuracy', 'auc', 'confusion_matrix', 'f1_score', 'log_loss', 'precision', 'recall', 'roc_curve'])

In [19]:
results['confusion_matrix'].print_rows(12,3)

+------------------+------------------+-------+
|   target_label   | predicted_label  | count |
+------------------+------------------+-------+
|   DiseaseClass   | SpecificDisease  |  120  |
| SpecificDisease  | CompositeMention |   1   |
|     Modifier     | SpecificDisease  |   47  |
| SpecificDisease  | SpecificDisease  |  241  |
|     Modifier     | CompositeMention |   1   |
|   DiseaseClass   |   DiseaseClass   |   19  |
| CompositeMention | SpecificDisease  |   23  |
| CompositeMention | CompositeMention |   2   |
+------------------+------------------+-------+
[8 rows x 3 columns]



In [33]:
pred_test = model.classify(test_data)

External memory mode: 1 batches

External memory mode: 1 batches

In [34]:
pred_test.export_csv('predictions_multiclass.csv')

In [ ]:
results.export_csv('multiclass_predictions.csv')

In [31]:
test_data.export_csv('multiclass_testdata.csv')